# District Cleaning — Jammu And Kashmir

This notebook standardizes **district names** for this state across:
- Enrolment data
- Demographic update data
- Biometric update data

**All data is saved back to the same cleaned files.**

In [18]:
import pandas as pd
from pathlib import Path

pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)

CLEAN_DIR = Path("../../data/processed/cleaned")

enrol_df = pd.read_csv(CLEAN_DIR / "enrolment_clean.csv")
demo_df  = pd.read_csv(CLEAN_DIR / "demographic_clean.csv")
bio_df   = pd.read_csv(CLEAN_DIR / "biometric_clean.csv")

for df in [enrol_df, demo_df, bio_df]:
    df["state"] = df["state"].astype(str).str.strip().str.title()
    df["district"] = df["district"].astype(str).str.strip().str.title()

print("✅ All datasets loaded and normalized (Title Case)")


✅ All datasets loaded and normalized (Title Case)


In [22]:
STATE_NAME = "Jammu And Kashmir"

districts = sorted(
    set(
        enrol_df.loc[enrol_df["state"] == STATE_NAME, "district"].dropna()
        .tolist()
    )
)

print(f"State: {STATE_NAME}")
print(f"Number of unique districts: {len(districts)}")

pd.DataFrame(
    {"District Name": districts}
)


State: Jammu And Kashmir
Number of unique districts: 20


,District Name
0,Anantnag
1,Bandipora
2,Baramulla
3,Budgam
4,Doda
5,Ganderbal
6,Jammu
7,Kathua
8,Kishtwar
9,Kulgam


## District Mapping

Add mappings in **Title Case only**.

Format:
```python
DISTRICT_MAPPING = {
    "Correct District": ["Wrong Name 1", "Wrong Name 2"],
}
```

In [20]:
DISTRICT_MAPPING = {
    "Budgam": ["Badgam"],
    "Bandipora": ["Bandipore","Bandipur"],
    "Baramulla": ["Baramula"],
    "Poonch": ["Punch"],
    "Rajouri": ["Rajauri"],
    "Shopian": ["Shupiyan"]   # "Correct District": ["Wrong Variant 1", "Wrong Variant 2"]
}

def apply_mapping(df, state, mapping, label):
    total = 0
    for correct, wrongs in mapping.items():
        mask = (
            (df["state"] == state) &
            (df["district"].isin(wrongs))
        )
        count = mask.sum()
        df.loc[mask, "district"] = correct
        total += count
        if count > 0:
            print(f"✔ {label} → {correct} : {count} rows fixed")
    return total

total_fixes = 0
total_fixes += apply_mapping(enrol_df, STATE_NAME, DISTRICT_MAPPING, "Enrolment")
total_fixes += apply_mapping(demo_df,  STATE_NAME, DISTRICT_MAPPING, "Demographic")
total_fixes += apply_mapping(bio_df,   STATE_NAME, DISTRICT_MAPPING, "Biometric")

print(f"✅ Total fixes in {STATE_NAME}: {total_fixes}")


✔ Enrolment → Budgam : 363 rows fixed
✔ Enrolment → Bandipora : 260 rows fixed
✔ Enrolment → Baramulla : 782 rows fixed
✔ Enrolment → Poonch : 273 rows fixed
✔ Enrolment → Rajouri : 2 rows fixed
✔ Enrolment → Shopian : 68 rows fixed
✔ Demographic → Budgam : 529 rows fixed
✔ Demographic → Bandipora : 318 rows fixed
✔ Demographic → Baramulla : 1113 rows fixed
✔ Demographic → Poonch : 285 rows fixed
✔ Demographic → Rajouri : 12 rows fixed
✔ Demographic → Shopian : 74 rows fixed
✅ Total fixes in Jammu And Kashmir: 4079


In [21]:
enrol_df.to_csv(CLEAN_DIR / "enrolment_clean.csv", index=False)
demo_df.to_csv(CLEAN_DIR / "demographic_clean.csv", index=False)
bio_df.to_csv(CLEAN_DIR / "biometric_clean.csv", index=False)

print("💾 All cleaned files saved successfully (overwritten)")


💾 All cleaned files saved successfully (overwritten)
